# <center><h1>Companhia Aberta Demonstrativo Financeiro</h1></center>

## <h2>Load Libraries</h2>

In [62]:
import pandas as pd
import numpy as np

## <h2>Import Data</h2>

<h3>Load .csv</h3>

In [128]:
cia_aberta_df = pd.read_csv(r'dfp_cia_aberta_BPA_con_2020.csv', encoding='ISO-8859-1', sep=";")    # leio o .csv com os dados de todas as companhias abertas

<h3>Select Company</h3>

In [129]:
df = cia_aberta_df[cia_aberta_df['CNPJ_CIA'] == '97.837.181/0001-47'].copy()    # dataframe = linhas do dataframe onde ('CNPJ_CIA' == '97.837.181/0001-47') == True
                                                                                    # (seleciono somente as linhas relativas a uma companhia de interesse)

In [130]:
df = df[['ORDEM_EXERC','CD_CONTA','DS_CONTA','VL_CONTA']]

In [131]:
df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']                                         # seleciono somente o último ou penúltimo exercício

In [132]:
df.reset_index(inplace=True, drop=True)

In [133]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
0,ÚLTIMO,1,Ativo Total,11498520.0
1,ÚLTIMO,1.01,Ativo Circulante,4220022.0
2,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,ÚLTIMO,1.01.02,Aplicações Financeiras,0.0
4,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...
71,ÚLTIMO,1.02.04.02.07,Goodwill na aquisição da Caetex Florestal,8767.0
72,ÚLTIMO,1.02.04.02.08,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,ÚLTIMO,1.02.04.02.09,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,ÚLTIMO,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


<h4>Subset DataFrame (only for testing purposes)</h4>

In [134]:
df = df.iloc[np.r_[0:7, 24:31],:]

In [135]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
0,ÚLTIMO,1,Ativo Total,11498520.0
1,ÚLTIMO,1.01,Ativo Circulante,4220022.0
2,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,ÚLTIMO,1.01.02,Aplicações Financeiras,0.0
4,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,ÚLTIMO,1.01.02.01.01,Títulos para Negociação,0.0
6,ÚLTIMO,1.01.02.01.02,Títulos Designados a Valor Justo,0.0
24,ÚLTIMO,1.02,Ativo Não Circulante,7278498.0
25,ÚLTIMO,1.02.01,Ativo Realizável a Longo Prazo,2071636.0
26,ÚLTIMO,1.02.01.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0


## <h2>Wrangling</h2>

### <h3>Number of Steps</h3>

In [136]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])

In [137]:
cd_conta_len = [len(lst) for lst in cd_conta_split]

In [138]:
num_levels = max(cd_conta_len)

In [139]:
num_levels

5

### <h3>ROUND 1</h3>

In [140]:
round=1

In [141]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
0,ÚLTIMO,1,Ativo Total,11498520.0
1,ÚLTIMO,1.01,Ativo Circulante,4220022.0
2,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,ÚLTIMO,1.01.02,Aplicações Financeiras,0.0
4,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,ÚLTIMO,1.01.02.01.01,Títulos para Negociação,0.0
6,ÚLTIMO,1.01.02.01.02,Títulos Designados a Valor Justo,0.0
24,ÚLTIMO,1.02,Ativo Não Circulante,7278498.0
25,ÚLTIMO,1.02.01,Ativo Realizável a Longo Prazo,2071636.0
26,ÚLTIMO,1.02.01.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0


<h4>Join</h4>

In [161]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])

In [162]:
cd_conta_col1 = [row[:round] for row in cd_conta_split]

Select rows with only one element:

In [170]:
cd_conta_len = [len(row) for row in cd_conta_split]

In [163]:
idx_keys = [idx for idx, element in enumerate(cd_conta_len) if element == round]        # round = 1. idx_list?

In [164]:
keys = [cd_conta_col1[idx] for idx in idx_keys]

In [165]:
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_keys)}

In [172]:
df.iloc[idx_keys]

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,ÚLTIMO,1,Ativo Total,Ativo Total,11498520.0


Join:

In [87]:
'''
ds_conta_1 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if row[0] == key ]
# Não faz muito sentido escrever for row in cd_conta_split if row[0] == key quando eu já tenho cd_conta_col_1
'''

In [159]:
ds_conta_1 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for cd in cd_conta_col1
              if key == cd]

In [167]:
ds_conta_1 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [168]:
df.insert(2, column='DS_CONTA_1', value=ds_conta_1)

In [169]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,ÚLTIMO,1,Ativo Total,Ativo Total,11498520.0
1,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
2,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
3,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
4,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,ÚLTIMO,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
6,ÚLTIMO,1.01.02.01.02,Ativo Total,Títulos Designados a Valor Justo,0.0
24,ÚLTIMO,1.02,Ativo Total,Ativo Não Circulante,7278498.0
25,ÚLTIMO,1.02.01,Ativo Total,Ativo Realizável a Longo Prazo,2071636.0
26,ÚLTIMO,1.02.01.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0


<h4>Drop rows</h4>

Select rows with only one element:

In [174]:
df.iloc[idx_keys]

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,ÚLTIMO,1,Ativo Total,Ativo Total,11498520.0


Check if there is at least another CD_CONTA starting with '1'

In [175]:
mask = [False if len == 1 else True for len in cd_conta_len]

In [176]:
any(row[0] == '1' for row in cd_conta_split[mask])

True

In [177]:
# cd_conta_split = cd_conta_split[mask]

In [178]:
df = df[mask]

In [179]:
df.reset_index(inplace=True, drop=True)

In [180]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
1,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
2,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
3,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,ÚLTIMO,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
5,ÚLTIMO,1.01.02.01.02,Ativo Total,Títulos Designados a Valor Justo,0.0
6,ÚLTIMO,1.02,Ativo Total,Ativo Não Circulante,7278498.0
7,ÚLTIMO,1.02.01,Ativo Total,Ativo Realizável a Longo Prazo,2071636.0
8,ÚLTIMO,1.02.01.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,ÚLTIMO,1.02.01.01.01,Ativo Total,Títulos Designados a Valor Justo,0.0


ROUND 2

In [181]:
round=2

In [182]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
1,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
2,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
3,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,ÚLTIMO,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
5,ÚLTIMO,1.01.02.01.02,Ativo Total,Títulos Designados a Valor Justo,0.0
6,ÚLTIMO,1.02,Ativo Total,Ativo Não Circulante,7278498.0
7,ÚLTIMO,1.02.01,Ativo Total,Ativo Realizável a Longo Prazo,2071636.0
8,ÚLTIMO,1.02.01.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,ÚLTIMO,1.02.01.01.01,Ativo Total,Títulos Designados a Valor Justo,0.0


<h4>Join</h4>

In [185]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])

In [186]:
cd_conta_col2 = [row[:round] for row in cd_conta_split]

Select rows with only 2 elements:

In [187]:
cd_conta_len = [len(lst) for lst in cd_conta_split]

In [188]:
idx_keys = [idx for idx, element in enumerate(cd_conta_len) if element == round]

In [189]:
idx_keys

[0, 6]

In [190]:
df.iloc[idx_keys]

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
6,ÚLTIMO,1.02,Ativo Total,Ativo Não Circulante,7278498.0


In [191]:
keys = [cd_conta_col2[idx] for idx in idx_keys]

In [193]:
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_keys)}

In [199]:
ds_conta_2 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [678]:
# ds_conta_2 = [df['DS_CONTA'][key_idx_dict[key]] for row in cd_conta_split for key in key_idx_dict if ".".join(row[:round]) == key]

In [200]:
df.insert(3, column='DS_CONTA_2', value=ds_conta_2)

In [201]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,Ativo Circulante,4220022.0
1,ÚLTIMO,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
2,ÚLTIMO,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
3,ÚLTIMO,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,ÚLTIMO,1.01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
5,ÚLTIMO,1.01.02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
6,ÚLTIMO,1.02,Ativo Total,Ativo Não Circulante,Ativo Não Circulante,7278498.0
7,ÚLTIMO,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,2071636.0
8,ÚLTIMO,1.02.01.01,Ativo Total,Ativo Não Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
9,ÚLTIMO,1.02.01.01.01,Ativo Total,Ativo Não Circulante,Títulos Designados a Valor Justo,0.0


<h4>Drop rows</h4>

Select rows with only two elements:

In [202]:
df.iloc[idx_keys]

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,Ativo Circulante,4220022.0
6,ÚLTIMO,1.02,Ativo Total,Ativo Não Circulante,Ativo Não Circulante,7278498.0


Check if there is at least another CD_CONTA starting with '1.01' or '1.02':

In [96]:
cd_conta_split_drop = cd_conta_split.copy()

In [685]:
# del cd_conta_split_drop[0,23]

In [241]:
any(row[1] == '01' for row in cd_conta_split_drop)

True

In [245]:
any(row[1] == '02' for row in cd_conta_split_drop)

True

In [263]:
cd_conta_split_3 = cd_conta_split_2[0+1:23] + cd_conta_split_2[23+1:]

In [264]:
cd_conta_split_3

[['01', '01'],
 ['01', '02'],
 ['01', '02', '01'],
 ['01', '02', '01', '01'],
 ['01', '02', '01', '02'],
 ['01', '02', '02'],
 ['01', '02', '03'],
 ['01', '03'],
 ['01', '03', '01'],
 ['01', '03', '01', '01'],
 ['01', '03', '01', '02'],
 ['01', '03', '02'],
 ['01', '03', '02', '01'],
 ['01', '04'],
 ['01', '05'],
 ['01', '06'],
 ['01', '06', '01'],
 ['01', '07'],
 ['01', '08'],
 ['01', '08', '01'],
 ['01', '08', '02'],
 ['01', '08', '03'],
 ['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '01', '01'],
 ['02', '01', '02'],
 ['02', '01', '03'],
 ['02', '01', '04'],
 ['02', '01', '04', '01'],
 ['02', '01', '04', '02'],
 ['02', '01', '05'],
 ['02', '01', '06'],
 ['02', '01', '07'],
 ['02', '01', '07', '01'],
 ['02', '01', '08'],
 ['02', '01', '09'],
 ['02', '01', '09', '01'],
 ['02', '01', '09', '03'],
 ['02', '01', '09', '04'],
 ['02', '01', '10'],
 ['02', '01', '10', '01'],
 ['02', '01', '10', '02'],
 ['02', '01', '10', '03'],
 ['02', '01', '10', '04'],
 ['02', '01', '10', '05'],
 ['02',

In [258]:
df = df.drop(df.index[[0,23]])

In [265]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
56650,ÚLTIMO,01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
56654,ÚLTIMO,01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
56658,ÚLTIMO,01.02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...,...
56788,ÚLTIMO,02.04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,02.04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,02.04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,02.04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


ROUND 3

In [266]:
cd_conta_split_3 = [row[1:] for row in cd_conta_split_3]

In [267]:
cd_conta_split_3

[['01'],
 ['02'],
 ['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '02'],
 ['02', '02'],
 ['02', '03'],
 ['03'],
 ['03', '01'],
 ['03', '01', '01'],
 ['03', '01', '02'],
 ['03', '02'],
 ['03', '02', '01'],
 ['04'],
 ['05'],
 ['06'],
 ['06', '01'],
 ['07'],
 ['08'],
 ['08', '01'],
 ['08', '02'],
 ['08', '03'],
 ['01'],
 ['01', '01'],
 ['01', '01', '01'],
 ['01', '02'],
 ['01', '03'],
 ['01', '04'],
 ['01', '04', '01'],
 ['01', '04', '02'],
 ['01', '05'],
 ['01', '06'],
 ['01', '07'],
 ['01', '07', '01'],
 ['01', '08'],
 ['01', '09'],
 ['01', '09', '01'],
 ['01', '09', '03'],
 ['01', '09', '04'],
 ['01', '10'],
 ['01', '10', '01'],
 ['01', '10', '02'],
 ['01', '10', '03'],
 ['01', '10', '04'],
 ['01', '10', '05'],
 ['01', '10', '06'],
 ['02'],
 ['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '04'],
 ['02', '01', '05'],
 ['02', '02'],
 ['03'],
 ['03', '01'],
 ['03', '02'],
 ['03', '03'],
 ['04'],
 ['04', '01'],
 ['04', '01', '01'],
 ['04', '01', '02'],
 ['04', '01', '03'],
 ['04', '02']

In [269]:
[".".join(row) for row in cd_conta_split_3]

['01',
 '02',
 '02.01',
 '02.01.01',
 '02.01.02',
 '02.02',
 '02.03',
 '03',
 '03.01',
 '03.01.01',
 '03.01.02',
 '03.02',
 '03.02.01',
 '04',
 '05',
 '06',
 '06.01',
 '07',
 '08',
 '08.01',
 '08.02',
 '08.03',
 '01',
 '01.01',
 '01.01.01',
 '01.02',
 '01.03',
 '01.04',
 '01.04.01',
 '01.04.02',
 '01.05',
 '01.06',
 '01.07',
 '01.07.01',
 '01.08',
 '01.09',
 '01.09.01',
 '01.09.03',
 '01.09.04',
 '01.10',
 '01.10.01',
 '01.10.02',
 '01.10.03',
 '01.10.04',
 '01.10.05',
 '01.10.06',
 '02',
 '02.01',
 '02.01.01',
 '02.01.04',
 '02.01.05',
 '02.02',
 '03',
 '03.01',
 '03.02',
 '03.03',
 '04',
 '04.01',
 '04.01.01',
 '04.01.02',
 '04.01.03',
 '04.02',
 '04.02.01',
 '04.02.02',
 '04.02.03',
 '04.02.04',
 '04.02.05',
 '04.02.06',
 '04.02.07',
 '04.02.08',
 '04.02.09',
 '04.02.10',
 '04.02.11']

In [270]:
df['CD_CONTA'] = [".".join(row) for row in cd_conta_split_3]

In [271]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
56650,ÚLTIMO,01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
56654,ÚLTIMO,02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
56658,ÚLTIMO,02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...,...
56788,ÚLTIMO,04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


In [272]:
third_column = [row[0] for row in cd_conta_split_3]

In [274]:
CD_DS_CONTA_dict = {cd: ds for cd, ds in zip(df['CD_CONTA'], df['DS_CONTA'])}

In [275]:
def join(key):
    ds_conta = CD_DS_CONTA_dict[key]
    return ds_conta

In [276]:
ds_conta_3 = [join(key) for key in third_column]

In [277]:
ds_conta_3

['Ativo Realizável a Longo Prazo',
 'Investimentos',
 'Investimentos',
 'Investimentos',
 'Investimentos',
 'Investimentos',
 'Investimentos',
 'Imobilizado',
 'Imobilizado',
 'Imobilizado',
 'Imobilizado',
 'Imobilizado',
 'Imobilizado',
 'Intangível',
 'Ativos Biológicos',
 'Tributos a Recuperar',
 'Tributos a Recuperar',
 'Despesas Antecipadas',
 'Outros Ativos Circulantes',
 'Outros Ativos Circulantes',
 'Outros Ativos Circulantes',
 'Outros Ativos Circulantes',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'At

In [278]:
df.insert(4, column='DS_CONTA_3', value=ds_conta_3)

In [279]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
56650,ÚLTIMO,01,Ativo Total,Ativo Circulante,Ativo Realizável a Longo Prazo,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,02,Ativo Total,Ativo Circulante,Investimentos,Aplicações Financeiras,0.0
56654,ÚLTIMO,02.01,Ativo Total,Ativo Circulante,Investimentos,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,02.01.01,Ativo Total,Ativo Circulante,Investimentos,Títulos para Negociação,0.0
56658,ÚLTIMO,02.01.02,Ativo Total,Ativo Circulante,Investimentos,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...,...,...
56788,ÚLTIMO,04.02.07,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,04.02.08,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,04.02.09,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,04.02.10,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0
